# 1 Data wrangling<a id='1_Data_wrangling'></a>

## 1.1 Contents<a id='1.1_Contents'></a>
* [1 Data wrangling](#1_Data_wrangling)
  * [1.1 Contents](#1.1_Contents)
  * [1.2 Introduction](#1.2_Introduction)
  * [1.3 Imports](#1.3_Imports)
  * [1.4 Load The Data](#1.4_Load_The_Data)
  * [1.5 Explore The Data](#1.5_Explore_The_Data)
    * [1.5.1 Number Of Missing Values By Column](#1.5.1_Number_Of_Missing_Values_By_Column)
    * [1.5.2 Correcting Feature Types](#1.5.2_Correcting_Feature_Types)
    * [1.5.3 Distribution of Null Values](#1.5.3_Distribution_of_Null_Values)
  * [1.6 Imputing Null Values](#1.6_Imputing_Null_Values)

## 1.2 Introduction<a id='1.2_Introduction'></a>

The first step will be collecting data, organizing it, and making sure it's well defined.

## 1.3 Imports<a id='1.3_Imports'></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

## 1.4 Load The Data<a id='1.4_Load_The_Data'></a>

In [2]:
loan_default_data = pd.read_csv('../data/raw/loan_default_data.csv')

In [3]:
loan_default_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38480 entries, 0 to 38479
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              38480 non-null  int64  
 1   id                      38480 non-null  int64  
 2   member_id               38480 non-null  int64  
 3   loan_amnt               38479 non-null  float64
 4   funded_amnt             38479 non-null  float64
 5   funded_amnt_inv         38479 non-null  float64
 6   term                    38480 non-null  object 
 7   int_rate                38480 non-null  float64
 8   installment             38479 non-null  float64
 9   emp_length              37487 non-null  object 
 10  home_ownership          38480 non-null  object 
 11  annual_inc              38478 non-null  float64
 12  verification_status     38480 non-null  object 
 13  issue_d                 38480 non-null  object 
 14  loan_status             38480 non-null

In [4]:
loan_default_data.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,...,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,repay_fail
0,2,2,2,0.0,0.0,0.0,36 months,0.00,0.00,< 1 year,...,1.0,0.000000,0.00,0.00,0.00,7-Jan,0.00,7-Jan,7-Jan,1
1,3,545583,703644,2500.0,2500.0,2500.0,36 months,13.98,85.42,4 years,...,10.0,3075.291779,3075.29,2500.00,575.29,13-Jul,90.85,13-Aug,16-Jun,0
2,4,532101,687836,5000.0,5000.0,5000.0,36 months,15.95,175.67,4 years,...,15.0,2948.760000,2948.76,1909.02,873.81,11-Nov,175.67,NaN,12-Mar,1
3,5,877788,1092507,7000.0,7000.0,7000.0,36 months,9.91,225.58,10+ years,...,20.0,8082.391880,8082.39,7000.00,1082.39,14-Mar,1550.27,NaN,14-Mar,0
4,6,875406,1089981,2000.0,2000.0,2000.0,36 months,5.42,60.32,10+ years,...,15.0,2161.663244,2161.66,2000.00,161.66,14-Feb,53.12,NaN,16-Jun,0


## 1.5 Explore The Data<a id='1.5_Explore_The_Data'></a>

### 1.5.1 Number Of Missing Values By Column<a id='1.5.1_Number_Of_Missing_Values_By_Column'></a>

In [5]:
missing = pd.concat([loan_default_data.isnull().sum(), 100 * loan_default_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count'], ascending = False)

,count,%
next_pymnt_d,35097,91.208420
mths_since_last_delinq,24363,63.313410
emp_length,993,2.580561
last_pymnt_d,71,0.184511
revol_util,59,0.153326
revol_bal,4,0.010395
last_credit_pull_d,3,0.007796
annual_inc,2,0.005198
installment,1,0.002599
last_pymnt_amnt,1,0.002599


There are only two columns that are missing a majority of their features. `next_pymnt_d` (next payment date) is missing over 90% of its entries. Fortunately, with our objective being predicting the likelihood of a loan default, this feature is likely not significant in determining a default. The same goes for `mths_since_last_delinq` (months since last delinquency), which likely holds more power in determining default; however, the other features included in the data will surely be more than enough. The good news is that the next largest percentage of missing data is just over 2% for `emp_length` (employment length). Our data looks very promising to utilize.

### 1.5.2 Correcting Feature Types<a id='1.5.2_Correcting_Feature_Types'></a>

It is worth noting that some of the columns in the data are categorical although numeric, such as `repay_fail` (whether the client defaulted or not) and the .info() method has captured the feature as an integer data type. Similarly, there are some that are indetified as object type by pandas, but can be represented numerically, such as `term`. We'll have to go in and correct the data types for the features. We will also check for the unique values to see if most of the column consists of one value, making feature not good for predicting.

In [6]:
#Checking what values to replace with numeric data
loan_default_data['term'].value_counts()

36 months    28593
60 months     9887
Name: term, dtype: int64

In [7]:
#Saving the corrected column to a new data frame
loan_default_corrected_types = loan_default_data.copy()
loan_default_corrected_types['term'] = loan_default_corrected_types['term'].replace({'36 months': 36, '60 months': 60})
loan_default_corrected_types['term'].unique()

array([36, 60], dtype=int64)

In [8]:
#Employment length may be best kept in this format because it accounts for < 1 year and +10 years
loan_default_corrected_types['emp_length'].unique()

array(['< 1 year', '4 years', '10+ years', nan, '3 years', '5 years',
       '7 years', '2 years', '1 year', '6 years', '9 years', '8 years'],
      dtype=object)

We'll inspect the features that didn't fit with the previous inspection and see which ones could potentially have their types corrected as well.

In [9]:
loan_default_corrected_types.loc[:,'home_ownership':'revol_util'].head()

,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util
0,RENT,0.0,Not Verified,Dec-99,Charged Off,major_purchase,000xx,ZZ,0.00,0.0,1-Jan,0.0,NaN,0.0,0.0,0.0,“0.00%”
1,RENT,20004.0,Not Verified,10-Jul,Does not meet the credit policy. Status:Fully ...,other,487xx,MI,19.86,0.0,5-Aug,5.0,NaN,7.0,0.0,981.0,21.30%
2,RENT,59000.0,Not Verified,10-Jun,Charged Off,debt_consolidation,115xx,NY,19.57,0.0,Apr-94,1.0,59.0,7.0,0.0,18773.0,99.90%
3,MORTGAGE,53796.0,Not Verified,11-Sep,Fully Paid,other,751xx,TX,10.80,3.0,Mar-98,3.0,3.0,7.0,0.0,3269.0,47.20%
4,RENT,30000.0,Not Verified,11-Sep,Fully Paid,debt_consolidation,112xx,NY,3.60,0.0,Jan-75,0.0,72.0,7.0,0.0,0.0,0%


In [10]:
loan_default_corrected_types['home_ownership'].value_counts()

RENT        18253
MORTGAGE    17140
OWN          2958
OTHER         125
NONE            4
Name: home_ownership, dtype: int64

In [11]:
loan_default_corrected_types['verification_status'].value_counts()

Not Verified       16962
Verified           12170
Source Verified     9348
Name: verification_status, dtype: int64

In [12]:
loan_default_corrected_types['loan_status'].value_counts()

Fully Paid                                             29987
Charged Off                                             5111
Does not meet the credit policy. Status:Fully Paid      1782
Current                                                  867
Does not meet the credit policy. Status:Charged Off      689
Late (31-120 days)                                        20
In Grace Period                                           15
Late (16-30 days)                                          7
Default                                                    2
Name: loan_status, dtype: int64

We can consolidate the long strings into their respective categories of 'Fully Paid' and 'Charged off'.

In [13]:
condition_1 = loan_default_corrected_types['loan_status'].str.contains('Fully Paid') 
condition_2 = loan_default_corrected_types['loan_status'].str.contains('Charged Off') 

loan_default_corrected_types.loc[condition_1, 'loan_status'] = 'Fully Paid'
loan_default_corrected_types.loc[condition_2, 'loan_status'] = 'Charged Off'

loan_default_corrected_types['loan_status'].value_counts()

Fully Paid            31769
Charged Off            5800
Current                 867
Late (31-120 days)       20
In Grace Period          15
Late (16-30 days)         7
Default                   2
Name: loan_status, dtype: int64

As a sanity check, we can see if the number of defaults and charged off loans match the defaults in the `repay_fail` column.

In [14]:
loan_default_corrected_types['repay_fail'].sum()

5829

`repay_fail` is also counting the 'Late (31-120 days)' and 'Late (16-30 days)' categories. Therefore the only loans not counted as default are the following categories: 'Fully Paid', 'Current', 'In Grace Period'. This is good news since the data is consistent in that manner.

Continuing with correcting the data types.

In [15]:
loan_default_corrected_types['purpose'].value_counts()

debt_consolidation    17917
credit_card            4974
other                  3950
home_improvement       2901
major_purchase         2079
small_business         1808
car                    1481
wedding                 909
medical                 675
moving                  562
house                   387
educational             386
vacation                360
renewable_energy         91
Name: purpose, dtype: int64

For the `addr_state` (address state), there should be fifty states, we can check for consistency, although there should be at least 51 as the result because the first column shows 'ZZ' as one of the states, surely to represent a null value.

In [16]:
loan_default_corrected_types['addr_state'].nunique()

52

In [17]:
loan_default_corrected_types['addr_state'].unique()

array(['ZZ', 'MI', 'NY', 'TX', 'AL', 'AZ', 'MO', 'OH', 'CA', 'AR', 'NJ',
       'WI', 'VA', 'GA', 'FL', 'AK', 'MA', 'MN', 'MD', 'NV', 'KY', 'IL',
       'AA', 'NH', 'CT', 'WA', 'NC', 'SD', 'UT', 'OR', 'DC', 'WY', 'KS',
       'PA', 'SC', 'RI', 'HI', 'LA', 'CO', 'WV', 'OK', 'VT', 'MT', 'NM',
       'DE', 'TN', 'MS', 'NE', 'IN', 'ID', 'IA', 'ME'], dtype=object)

We can see that 'ZZ' and 'AA' are likely acting as placeholders for null values.

From inspecting `revol_util` (revolving debt limit utilized), we see that it should be a percentage but it is listed as object type. From the first row we can see there are quotation marks around the null value for `revol_util`. We'll strip the strings from anything non-numeric and convert them to floats.

In [18]:
corrected_percentages = loan_default_corrected_types['revol_util'].astype(str).str.replace('[^0-9.]', '', regex=True)

loan_default_corrected_types['revol_util'] = pd.to_numeric(corrected_percentages, errors='coerce')

loan_default_corrected_types['revol_util'].head()

0     0.0
1    21.3
2    99.9
3    47.2
4     0.0
Name: revol_util, dtype: float64

All the rest of the features fall in line to their respective types.

### 1.5.3 Distribution of Null Values<a id='1.5.3_Distribution_of_Null_Values'></a>

Before getting into removing null values that aren't needed, we should make sure null values aren't mostly exclusive to the `repay_fail` column, which is what we will want to be predicting. We'll split up the data into defaulted loans and non-defaulted loans and compare how many null values are in each set of data.

In [19]:
loan_defaulted = loan_default_corrected_types[loan_default_corrected_types['repay_fail'] == 1]
loan_not_defaulted = loan_default_corrected_types[loan_default_corrected_types['repay_fail'] == 0]

In [20]:
missing = pd.concat([loan_defaulted.isnull().sum(), 100 * loan_defaulted.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count'], ascending = False)

,count,%
next_pymnt_d,5110,87.665123
mths_since_last_delinq,3445,59.101046
emp_length,219,3.757077
last_pymnt_d,71,1.218048
revol_util,19,0.325956
annual_inc,1,0.017156
last_credit_pull_d,1,0.017156
revol_bal,1,0.017156
open_acc,0,0.000000
pub_rec,0,0.000000


In [21]:
missing = pd.concat([loan_not_defaulted.isnull().sum(), 100 * loan_not_defaulted.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count'], ascending = False)

,count,%
next_pymnt_d,29987,91.840985
mths_since_last_delinq,20918,64.065419
emp_length,774,2.370525
revol_util,40,0.122508
revol_bal,3,0.009188
last_credit_pull_d,2,0.006125
total_rec_prncp,1,0.003063
inq_last_6mths,1,0.003063
pub_rec,1,0.003063
delinq_2yrs,1,0.003063


`last_pymnt_d` (Last payment date) has more missing values for loans defaulted, which makes sense if they are defaulting on their loans and haven't made a payment at all. `emp_length` has a higher percentage of missing values for loans defaulted, which is also intuitive because they likely were unemployed (hence, the missing value) and are more likely to default in that scenario. The other null values are distributed more equal, which is a good sign that the missing values aren't particularly exlusive to the defaulted loans.

## 1.6 Removing Data

### 1.6.1 Irrelevant Features

The purpose of the model will be to predict if a borrower is likely to default on a loan. The inputs of the model will then have to be based on features with information gathered at the application process. There are features included in the data that pertain to information gathered after the loan was funded. This is data that we can therefore remove. 

In [25]:
#Creating the variable for the new DataFrame loan_default_2 that will be the data without the irrelevant features

columns = ['loan_amnt','term','int_rate','installment','emp_length','home_ownership','annual_inc','verification_status',
          'purpose','dti','delinq_2yrs','earliest_cr_line','inq_last_6mths','open_acc','pub_rec','revol_bal','revol_util',
          'total_acc','repay_fail']

loan_default_2 = loan_default_corrected_types[columns]

loan_default_2.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,repay_fail
0,0.0,36,0.00,0.00,< 1 year,RENT,0.0,Not Verified,major_purchase,0.00,0.0,1-Jan,0.0,0.0,0.0,0.0,0.0,1.0,1
1,2500.0,36,13.98,85.42,4 years,RENT,20004.0,Not Verified,other,19.86,0.0,5-Aug,5.0,7.0,0.0,981.0,21.3,10.0,0
2,5000.0,36,15.95,175.67,4 years,RENT,59000.0,Not Verified,debt_consolidation,19.57,0.0,Apr-94,1.0,7.0,0.0,18773.0,99.9,15.0,1
3,7000.0,36,9.91,225.58,10+ years,MORTGAGE,53796.0,Not Verified,other,10.80,3.0,Mar-98,3.0,7.0,0.0,3269.0,47.2,20.0,0
4,2000.0,36,5.42,60.32,10+ years,RENT,30000.0,Not Verified,debt_consolidation,3.60,0.0,Jan-75,0.0,7.0,0.0,0.0,0.0,15.0,0


### 1.6.2 Removing Null Values

As mentioned, employment length has many missing values, but could represent the unemployed. We can inspect the data of those who have missing values for `emp_length` and no annual income.

In [23]:
loan_default_corrected_types[loan_default_corrected_types['annual_inc'] == 0]

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,...,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,repay_fail
0,2,2,2,0.0,0.0,0.0,36,0.0,0.0,< 1 year,...,1.0,0.0,0.0,0.0,0.0,7-Jan,0.0,7-Jan,7-Jan,1


There is only one column that has no annual income, meaning that the employment length values are more likely just missing at random.